# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: Benjamin Leonardo Zarate Solano

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Examen_1")
    .master("spark://b631371550f9:7077")
    .config("spark.ui.port", "4040")
    .getOrCreate()
)
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 13:43:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read the Data (10 points):

In [7]:
from team_name.spark_utils import SparkUtils

employees_schema = SparkUtils.generate_schema(
    [
        ("employee_id", "integer"), 
        ("employee_info", "string")
    ]
)

departments_schema = SparkUtils.generate_schema(
    [
        ("department_id", "integer"),
        ("department_name", "string"),
        ("location", "string"),
    ]
)

employees_df = (spark.read \
                    .schema(employees_schema) \
                    .option("header", "true") \
                    .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/employees.csv")
)

departments_df = (spark.read \
                    .schema(departments_schema) \
                    .option("header", "true") \
                    .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/departments.csv")
)

employees_df.printSchema()
departments_df.printSchema()

employees_df.show(5, truncate=False)
departments_df.show(5, truncate=False)

root
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)

root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)

+-----------+-------------------------------------------------------------------------------------------+
|employee_id|employee_info                                                                              |
+-----------+-------------------------------------------------------------------------------------------+
|1          |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |
|2          |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |
|3          |{'name': 'Carrie', 'department_id': 105, 'salary': 114421.44, 'hire_date': '1998-12-10'}   |
|4          |{'name': 'Renee', 'department_id': 104, 'salary': 54688.13, 'hire_date': '1995-03-17'}     |
|5          |{'name': 'G

### Extract the employee info from the JSON column (20 points):

In [8]:
from pyspark.sql.functions import get_json_object

employees_df = employees_df.withColumn("name", get_json_object(employees_df.employee_info, "$.name")) \
                           .withColumn("department_id", get_json_object(employees_df.employee_info, "$.department_id")) \
                           .withColumn("salary", get_json_object(employees_df.employee_info, "$.salary")) \
                           .withColumn("hire_date", get_json_object(employees_df.employee_info, "$.hire_date"))

employees_df.select("employee_id", "name", "department_id", "salary", "hire_date").show(truncate=False)

+-----------+---------+-------------+---------+----------+
|employee_id|name     |department_id|salary   |hire_date |
+-----------+---------+-------------+---------+----------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|
|2          |Rachel   |104          |100820.16|2009-07-01|
|3          |Carrie   |105          |114421.44|1998-12-10|
|4          |Renee    |104          |54688.13 |1995-03-17|
|5          |Gabriella|109          |106267.03|1995-02-09|
|6          |Kristen  |101          |88237.54 |2010-11-15|
|7          |Jonathan |102          |39323.42 |2012-06-30|
|8          |Michelle |101          |64262.85 |2005-10-30|
|9          |Michelle |105          |103521.88|1991-07-10|
|10         |Lisa     |110          |55435.93 |2016-03-25|
|11         |Cheryl   |103          |88073.75 |2020-08-21|
|12         |Mikayla  |107          |95192.05 |2022-05-13|
|13         |Lisa     |104          |36032.49 |2019-05-16|
|14         |Sean     |108          |64904.69 |2021-06-0

### Join Operations (10 points):

In [9]:
employees_department_join = employees_df.join(departments_df,employees_df["department_id"] == departments_df["department_id"],"inner",)

employees_department_join.show()

+-----------+--------------------+---------+-------------+---------+----------+-------------+--------------------+-------------+
|employee_id|       employee_info|     name|department_id|   salary| hire_date|department_id|     department_name|     location|
+-----------+--------------------+---------+-------------+---------+----------+-------------+--------------------+-------------+
|          1|{'name': 'Caitlyn...|  Caitlyn|          103|115959.78|2002-06-10|          103| Sales and Marketing|      Chicago|
|          2|{'name': 'Rachel'...|   Rachel|          104|100820.16|2009-07-01|          104|    Data Engineering|      Zapopan|
|          3|{'name': 'Carrie'...|   Carrie|          105|114421.44|1998-12-10|          105|        Data Science|      Seattle|
|          4|{'name': 'Renee',...|    Renee|          104| 54688.13|1995-03-17|          104|    Data Engineering|      Zapopan|
|          5|{'name': 'Gabriel...|Gabriella|          109|106267.03|1995-02-09|          109|    

### Transformations (Using when()) (10 points):

In [ ]:
from pyspark.sql.functions import when, col

employees_department_join = emp_dept_df.withColumn(
    "salary_category", when(col("salary") >= 55000, "High").otherwise("Low")
)

emp_dept_df.select("name", "salary", "salary_category").show(5)

### Filter and Group (20 points):

### Sort (10 points):  

### Find the number of employees with more years in the company (15 points).

### Display the  Spark Plan (5 points):